In [1]:

# imports
import os
import sys
import types
import json
import base64

# figure size/format
fig_width = 7
fig_height = 5
fig_format = 'retina'
fig_dpi = 96
interactivity = ''
is_shiny = False
is_dashboard = False
plotly_connected = True

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = "figure"
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  if plotly_connected:
    pio.renderers.default = "notebook_connected"
  else:
    pio.renderers.default = "notebook"
  for template in pio.templates.keys():
    pio.templates[template].layout.margin = dict(t=30,r=0,b=0,l=0)
except Exception:
  pass

# disable itables paging for dashboards
if is_dashboard:
  try:
    from itables import options
    options.dom = 'fiBrtlp'
    options.maxBytes = 1024 * 1024
    options.language = dict(info = "Showing _TOTAL_ entries")
    options.classes = "display nowrap compact"
    options.paging = False
    options.searching = True
    options.ordering = True
    options.info = True
    options.lengthChange = False
    options.autoWidth = False
    options.responsive = True
    options.keys = True
    options.buttons = []
  except Exception:
    pass
  
  try:
    import altair as alt
    # By default, dashboards will have container sized
    # vega visualizations which allows them to flow reasonably
    theme_sentinel = '_quarto-dashboard-internal'
    def make_theme(name):
        nonTheme = alt.themes._plugins[name]    
        def patch_theme(*args, **kwargs):
            existingTheme = nonTheme()
            if 'height' not in existingTheme:
              existingTheme['height'] = 'container'
            if 'width' not in existingTheme:
              existingTheme['width'] = 'container'

            if 'config' not in existingTheme:
              existingTheme['config'] = dict()
            
            # Configure the default font sizes
            title_font_size = 15
            header_font_size = 13
            axis_font_size = 12
            legend_font_size = 12
            mark_font_size = 12
            tooltip = False

            config = existingTheme['config']

            # The Axis
            if 'axis' not in config:
              config['axis'] = dict()
            axis = config['axis']
            if 'labelFontSize' not in axis:
              axis['labelFontSize'] = axis_font_size
            if 'titleFontSize' not in axis:
              axis['titleFontSize'] = axis_font_size  

            # The legend
            if 'legend' not in config:
              config['legend'] = dict()
            legend = config['legend']
            if 'labelFontSize' not in legend:
              legend['labelFontSize'] = legend_font_size
            if 'titleFontSize' not in legend:
              legend['titleFontSize'] = legend_font_size  

            # The header
            if 'header' not in config:
              config['header'] = dict()
            header = config['header']
            if 'labelFontSize' not in header:
              header['labelFontSize'] = header_font_size
            if 'titleFontSize' not in header:
              header['titleFontSize'] = header_font_size    

            # Title
            if 'title' not in config:
              config['title'] = dict()
            title = config['title']
            if 'fontSize' not in title:
              title['fontSize'] = title_font_size

            # Marks
            if 'mark' not in config:
              config['mark'] = dict()
            mark = config['mark']
            if 'fontSize' not in mark:
              mark['fontSize'] = mark_font_size

            # Mark tooltips
            if tooltip and 'tooltip' not in mark:
              mark['tooltip'] = dict(content="encoding")

            return existingTheme
            
        return patch_theme

    # We can only do this once per session
    if theme_sentinel not in alt.themes.names():
      for name in alt.themes.names():
        alt.themes.register(name, make_theme(name))
      
      # register a sentinel theme so we only do this once
      alt.themes.register(theme_sentinel, make_theme('default'))
      alt.themes.enable('default')

  except Exception:
    pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass

# interactivity
if interactivity:
  from IPython.core.interactiveshell import InteractiveShell
  InteractiveShell.ast_node_interactivity = interactivity

# NOTE: the kernel_deps code is repeated in the cleanup.py file
# (we can't easily share this code b/c of the way it is run).
# If you edit this code also edit the same code in cleanup.py!

# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
run_path = 'L1VzZXJzL3BoaWxjaG9kcm93L015IERyaXZlIChwY2hvZHJvd0BtaWRkbGVidXJ5LmVkdSkvdGVhY2hpbmcvbWwtbm90ZXMvc291cmNl'
if run_path:
  # hex-decode the path
  run_path = base64.b64decode(run_path.encode("utf-8")).decode("utf-8")
  os.chdir(run_path)

# reset state
%reset

# shiny
# Checking for shiny by using False directly because we're after the %reset. We don't want
# to set a variable that stays in global scope.
if False:
  try:
    import htmltools as _htmltools
    import ast as _ast

    _htmltools.html_dependency_render_mode = "json"

    # This decorator will be added to all function definitions
    def _display_if_has_repr_html(x):
      try:
        # IPython 7.14 preferred import
        from IPython.display import display, HTML
      except:
        from IPython.core.display import display, HTML

      if hasattr(x, '_repr_html_'):
        display(HTML(x._repr_html_()))
      return x

    # ideally we would undo the call to ast_transformers.append
    # at the end of this block whenver an error occurs, we do 
    # this for now as it will only be a problem if the user 
    # switches from shiny to not-shiny mode (and even then likely
    # won't matter)
    import builtins
    builtins._display_if_has_repr_html = _display_if_has_repr_html

    class _FunctionDefReprHtml(_ast.NodeTransformer):
      def visit_FunctionDef(self, node):
        node.decorator_list.insert(
          0,
          _ast.Name(id="_display_if_has_repr_html", ctx=_ast.Load())
        )
        return node

      def visit_AsyncFunctionDef(self, node):
        node.decorator_list.insert(
          0,
          _ast.Name(id="_display_if_has_repr_html", ctx=_ast.Load())
        )
        return node

    ip = get_ipython()
    ip.ast_transformers.append(_FunctionDefReprHtml())

  except:
    pass

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v

  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define
globals()["__spec__"] = None

/var/folders/xn/wvbwvw0d6dx46h9_2bkrknnw0000gn/T/ipykernel_2855/2649104261.py:25: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats(fig_format)


{"/opt/homebrew/Cellar/python@3.10/3.10.10/Frameworks/Python.framework/Versions/3.10/lib/python3.10/importlib/_bootstrap.py": 1675834351.0, "/opt/homebrew/Cellar/python@3.10/3.10.10/Frameworks/Python.framework/Versions/3.10/lib/python3.10/importlib/_bootstrap_external.py": 1675834351.0, "/opt/homebrew/Cellar/python@3.10/3.10.10/Frameworks/Python.framework/Versions/3.10/lib/python3.10/codecs.py": 1675834351.0, "/opt/homebrew/Cellar/python@3.10/3.10.10/Frameworks/Python.framework/Versions/3.10/lib/python3.10/encodings/aliases.py": 1675834351.0, "/opt/homebrew/Cellar/python@3.10/3.10.10/Frameworks/Python.framework/Versions/3.10/lib/python3.10/encodings/__init__.py": 1675834351.0, "/opt/homebrew/Cellar/python@3.10/3.10.10/Frameworks/Python.framework/Versions/3.10/lib/python3.10/encodings/utf_8.py": 1675834351.0, "/opt/homebrew/Cellar/python@3.10/3.10.10/Frameworks/Python.framework/Versions/3.10/lib/python3.10/abc.py": 1675834351.0, "/opt/homebrew/Cellar/python@3.10/3.10.10/Frameworks/Pytho

In [2]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

# !pip install torchinfo if running in Colab
from torchinfo import summary

import pandas as pd
import numpy as np
import time

# for embedding visualization later
import plotly.express as px
import plotly.io as pio

# for appearance
pio.templates.default = "plotly_white"

# for train-test split
from sklearn.model_selection import train_test_split

# for suppressing bugged warnings from torchinfo
import warnings
warnings.filterwarnings("ignore", category = UserWarning)

# tokenizers from HuggingFace
from transformers import BertTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# access the data
url = "https://raw.githubusercontent.com/PhilChodrow/PIC16B/master/datasets/news/News_Category_Dataset_v2.json"
df  = pd.read_json(url, lines=True)
df  = df[["category", "headline"]]

In [4]:
#---
df.head()
#---

,category,headline
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...


In [5]:
#---
df.groupby("category").size()
#---

category
ARTS               1509
ARTS & CULTURE     1339
BLACK VOICES       4528
BUSINESS           5937
COLLEGE            1144
COMEDY             5175
CRIME              3405
CULTURE & ARTS     1030
DIVORCE            3426
EDUCATION          1004
ENTERTAINMENT     16058
ENVIRONMENT        1323
FIFTY              1401
FOOD & DRINK       6226
GOOD NEWS          1398
GREEN              2622
HEALTHY LIVING     6694
HOME & LIVING      4195
IMPACT             3459
LATINO VOICES      1129
MEDIA              2815
MONEY              1707
PARENTING          8677
PARENTS            3955
POLITICS          32739
QUEER VOICES       6314
RELIGION           2556
SCIENCE            2178
SPORTS             4884
STYLE              2254
STYLE & BEAUTY     9649
TASTE              2096
TECH               2082
THE WORLDPOST      3664
TRAVEL             9887
WEDDINGS           3651
WEIRD NEWS         2670
WELLNESS          17827
WOMEN              3490
WORLD NEWS         2177
WORLDPOST          2579
dtype: 

In [6]:
categories = {
    "STYLE"   : 0,
    "SCIENCE" : 1,
    "TECH" : 2, 
}

df = df[df["category"].apply(lambda x: x in categories.keys())]
df.head()

,category,headline
137,TECH,Facebook Accused Of Reading Texts And Accessin...
138,TECH,Self-Driving Uber In Fatal Accident Had 6 Seco...
155,SCIENCE,Scientists Turn To DNA Technology To Search Fo...
272,TECH,Instagram Is Adding A 'Mute' Button For The Sa...
285,SCIENCE,Unusual Asteroid Could Be An Interstellar Gues...


In [7]:
df["category"] = df["category"].apply(categories.get)
df

,category,headline
137,2,Facebook Accused Of Reading Texts And Accessin...
138,2,Self-Driving Uber In Fatal Accident Had 6 Seco...
155,1,Scientists Turn To DNA Technology To Search Fo...
272,2,Instagram Is Adding A 'Mute' Button For The Sa...
285,1,Unusual Asteroid Could Be An Interstellar Gues...
...,...,...
200844,2,Google+ Now Open for Teens With Some Safeguards
200845,2,Web Wars
200846,2,"First White House Chief Technology Officer, An..."
200847,2,Watch The Top 9 YouTube Videos Of The Week


In [8]:
df.groupby("category").size() / len(df)

category
0    0.346024
1    0.334357
2    0.319619
dtype: float64

In [9]:
df_train, df_val = train_test_split(df,shuffle = True, test_size = 0.2)

In [10]:
#---
tokenizer = BertTokenizer.from_pretrained("google-bert/bert-base-uncased")
#---

In [11]:
#---
tokenizer.tokenize("I love logistic regression!")
#---

['i', 'love', 'log', '##istic', 'regression', '!']

In [12]:
#---
encoded = tokenizer("I love logistic regression!")
encoded
#---

{'input_ids': [101, 1045, 2293, 8833, 6553, 26237, 999, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
#---
tokenizer.decode(encoded["input_ids"])
#---

'[CLS] i love logistic regression! [SEP]'

In [14]:
X_train_tokenized = tokenizer(list(df_train["headline"]))["input_ids"]

In [15]:
max_len = 50

def pad(l, max_len):
    assert len(l) <= max_len
    to_add = max_len - len(l)
    return l + [0]*to_add

def preprocess(df, tokenizer, max_len):
    X = tokenizer(list(df["headline"]))["input_ids"]
    X = [pad(t, max_len) for t in X]
    y = list(df["category"])
    return X, y

class TextDataFromDF(Dataset):
    def __init__(self, df):
        self.X, self.y = preprocess(df, tokenizer, 50)

    def __getitem__(self, ix):
        return self.X[ix], self.y[ix]

    def __len__(self):
        return len(self.y)

In [16]:
train_data = TextDataFromDF(df_train)
val_data   = TextDataFromDF(df_val)

In [17]:
#---
X, y = train_data[1]
print(X)
print(y)
#---

[101, 7344, 11358, 2089, 8328, 2422, 2006, 4785, 2689, 2006, 3011, 1010, 6529, 2360, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1


In [18]:
def collate(data):
    X = torch.tensor([d[0] for d in data])
    y = torch.tensor([d[1] for d in data])
    return X,y 

train_loader = DataLoader(train_data, batch_size=8, shuffle=True, collate_fn = collate)
val_loader = DataLoader(val_data, batch_size=8, shuffle=True, collate_fn = collate)

In [19]:
X, y = next(iter(train_loader))

In [20]:
X[:3]

tensor([[  101,  4911,  2039,  1998,  3048,  2006,  2003,  2004,  3019,  2000,
          2149,  2004,  4634,  1999,  2293,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  6796, 12170, 22669,  5078,  1037, 26530,  3797,  1998,  2111,
          2288,  2428,  9421,  2125,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  2729,  2005,  1037,  9880,  1029, 15295,  1997,  5190,  1997,
         12004,  5308,  3516,  3023,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,

In [21]:
y[:3]

tensor([1, 0, 1])

In [22]:
#---
class TextClassificationModel(nn.Module):

    def __init__(self,vocab_size, embedding_dim, max_len, num_class):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size+1, embedding_dim)
        self.fc   = nn.Linear(max_len*embedding_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return(x)
#---

In [23]:
vocab_size = len(tokenizer.vocab)
embedding_dim = 10
num_class = len(categories)

model = TextClassificationModel(vocab_size, embedding_dim, max_len, num_class).to(device)

In [24]:
summary(model, input_Size = (8, max_len))

Layer (type:depth-idx)                   Param #
TextClassificationModel                  --
├─Embedding: 1-1                         305,230
├─Linear: 1-2                            1,503
Total params: 306,733
Trainable params: 306,733
Non-trainable params: 0

In [25]:
optimizer = torch.optim.Adam(model.parameters(), lr=.1)
loss_fn = torch.nn.CrossEntropyLoss()

def train(dataloader):
    epoch_start_time = time.time()
    # keep track of some counts for measuring accuracy
    total_acc, total_count = 0, 0
    
    for X, y in dataloader:
        
        #---
        # zero gradients
        optimizer.zero_grad()
        # form prediction on batch
        predicted_label = model(X)
        # evaluate loss on prediction
        loss = loss_fn(predicted_label, y)
        # compute gradient
        loss.backward()
        # take an optimization step
        optimizer.step()
        #---
        
        # for printing accuracy
        total_acc   += (predicted_label.argmax(1) == y).sum().item()
        total_count += y.size(0)

    print(f'| epoch {epoch:3d} | train accuracy {total_acc/total_count:8.3f} | time: {time.time() - epoch_start_time:5.2f}s')

def accuracy(dataloader):

    total_acc, total_count = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            predicted_label = model(X)
            total_acc += (predicted_label.argmax(1) == y).sum().item()
            total_count += y.size(0)
    return total_acc/total_count

In [26]:
#---
EPOCHS = 20
for epoch in range(1, EPOCHS + 1):
    train(train_loader)
#---

| epoch   1 | train accuracy    0.612 | time:  0.68s


| epoch   2 | train accuracy    0.864 | time:  0.63s


| epoch   3 | train accuracy    0.921 | time:  0.67s


| epoch   4 | train accuracy    0.945 | time:  0.53s


| epoch   5 | train accuracy    0.959 | time:  0.64s


...

| epoch  16 | train accuracy    0.994 | time:  0.57s


| epoch  17 | train accuracy    0.991 | time:  0.57s


| epoch  18 | train accuracy    0.992 | time:  0.66s


| epoch  19 | train accuracy    0.995 | time:  0.59s


| epoch  20 | train accuracy    0.994 | time:  0.58s


In [27]:
#---
accuracy(val_loader)
#---

0.8257866462010744

In [28]:
#---
embedding_matrix = model.embedding.cpu().weight.data.numpy()
#---

In [29]:
tokens = list(tokenizer.vocab.keys())

In [30]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
weights = pca.fit_transform(embedding_matrix)

In [31]:
tokens.append(" ")
embedding_df = pd.DataFrame({
    'word' : tokens,
    'x0'   : weights[:,0],
    'x1'   : weights[:,1]
})

embedding_df

,word,x0,x1
0,[PAD],0.580007,-0.038987
1,[unused0],-0.034409,-0.007818
2,[unused1],0.166999,-0.826541
3,[unused2],0.369909,0.920200
4,[unused3],-0.578982,-0.489246
...,...,...,...
30518,##／,1.056981,0.033426
30519,##：,-0.076317,-1.407349
30520,##？,-0.126401,-0.767700
30521,##～,0.213449,-0.570737


In [32]:
fig = px.scatter(embedding_df,
                 x = "x0",
                 y = "x1",
                 size = list(np.ones(len(embedding_df))),
                 size_max = 10,
                 hover_name = "word")

fig.show()

In [33]:
feminine = ["she", "her", "woman", "female"]
masculine = ["he", "him", "man", "male"]

highlight_1 = ["strong", "powerful", "smart",     "thinking", "brave", "muscle", "rough", "work"]
highlight_2 = ["hot",    "sexy",     "beautiful", "shopping", "children", "thin", "pretty", "home"]

def gender_mapper(x):
    if x in feminine:
        return 1
    elif x in masculine:
        return 4
    elif x in highlight_1:
        return 3
    elif x in highlight_2:
        return 2
    else:
        return 0

embedding_df["highlight"] = embedding_df["word"].apply(gender_mapper)
embedding_df["size"]      = np.array(1.0 + 50*(embedding_df["highlight"] > 0))

#
sub_df = embedding_df[embedding_df["highlight"] > 0]

In [34]:
import plotly.express as px

fig = px.scatter(sub_df,
                 x = "x0",
                 y = "x1",
                 color = "highlight",
                 size = list(sub_df["size"]),
                 size_max = 10,
                 hover_name = "word",
                 text = "word")

fig.update_traces(textposition='top center')
fig.update_layout(coloraxis_showscale=False)

fig.show()